
# Worksheet 2b - Single-Layer Perceptron

This is the third in a series of companion worksheets for for Andrej Karpathy's [Neural Networks: Zero To Hero](https://karpathy.ai/zero-to-hero.html) videos.

It corresponds to roughly the second half of the second video in the series, named "[The spelled-out intro to language modeling: building makemore](https://www.youtube.com/watch?v=PaCmpygFfXo)".

The rest of the worksheets are listed in the README [here](https://github.com/Russ741/karpathy-nn-z2h/).

The overall objective of this worksheet is to write code that generates a word that is similar to a set of example words it is trained on.
It does so using a single-layer neural network.

### Prerequisite: Load worksheet utilities

The following cell imports [utility functions](https://github.com/Russ741/karpathy-nn-z2h/blob/main/worksheets/worksheet_utils.py) that this worksheet depends on.
If the file isn't already locally available (e.g. for Colab), it downloads it from GitHub.

In [ ]:
try:
  from worksheet_utils import *
except ModuleNotFoundError:
  import requests

  utils_url = "https://raw.githubusercontent.com/Russ741/karpathy-nn-z2h/main/worksheets/worksheet_utils.py"
  utils_local_filename = "worksheet_utils.py"

  response = requests.get(utils_url)
  with open(utils_local_filename, mode='wb') as localfile:
    localfile.write(response.content)

  from worksheet_utils import *

### Preamble: Load data

Objective: Write a function that:
 * Loads the remotely-hosted [names.txt](https://github.com/karpathy/makemore/blob/master/names.txt) file
([raw link](https://github.com/karpathy/makemore/raw/master/names.txt))
 * Returns a list of strings
   * Each string should be equal to the word from the corresponding line of names.txt
   * The strings should not include line-break characters

Note: In practice, the order of the strings in the returned list does not matter, but for the
test to pass, they should be in the same order in the list as in words.txt.

Hint: In the video, Karpathy has a local copy of words.txt.<br>
One way to fetch words.txt is to use a function from the [requests](https://pypi.org/project/requests/) library.

Video: [0:03:03](https://youtu.be/PaCmpygFfXo?t=183)

In [ ]:
# TODO: Implement solution here

In [ ]:
def test_words():
    if not isinstance(loaded_words, list):
        print(f"Expected words to be a list")
        return
    expect_eq("len(loaded_words)", len(loaded_words), 32033)
    expect_eq("loaded_words[0]", loaded_words[0], "emma")
    expect_eq("loaded_words[-1]", loaded_words[-1], "zzyzx")
    print("load_words looks good. Onwards!")
loaded_words = load_words()
test_words()

### Step 1: Generate bigrams

Objective: Populate the variable ```bigrams``` with a list of bigrams (2-element tuples) of adjacent characters in ```words```.

Treat the start and end of each word as the character '.'

Video: [0:06:24](https://youtu.be/PaCmpygFfXo?t=384) and [0:21:55](https://youtu.be/PaCmpygFfXo?t=1315)

In [ ]:
# TODO: Implement solution here

In [ ]:
def test_generate_bigrams():
    bigrams = generate_bigrams(loaded_words)
    if not isinstance(bigrams, list):
        print(f"Expected bigrams to be a list")
        return
    expect_eq("count of ('.', 'm') bigrams", bigrams.count(('.', 'm')), 2538)
    expect_eq("count of ('a', 'b') bigrams", bigrams.count(('a', 'b')), 541)
    expect_eq("count of ('s', '.') bigrams", bigrams.count(('s', '.')), 1169)
    print("generate_bigrams looks good. Onwards!")
test_generate_bigrams()

### Step 2: Map characters to indices

Objective: Write a function that takes the following arguments:
* a list of char, char tuples representing all of the bigrams in a word list

And returns:
* a dict (```stoi```) where
  * the key is a character from ```words``` (including '.' for start/end),
  * the value is a unique integer, and
  * all the values are in the range from 0 to ```len(stoi) - 1``` (no gaps)

We'll use these unique integers as an index to represent the characters in a Tensor in later steps

Note that for this list of words, the same value of ```stoi``` could be generated without looking at the words at all,
but simply by using all the lowercase letters and a period. This approach would be more efficient for this exercise,
but will not generalize well conceptually to more complex models in future exercises.

Video: [0:15:40](https://youtu.be/PaCmpygFfXo?t=940)

In [ ]:
# TODO: Implement solution here

In [ ]:
import string

def test_get_stoi():
    bigrams = [
        ('.', 'h'),
        ('h', 'i'),
        ('i', '.'),
        ('.', 'b'),
        ('b', 'y'),
        ('y', 'e'),
        ('e', '.'),
    ]
    expected_s = sorted(['.', 'h', 'i', 'b', 'y', 'e'])
    stoi = get_stoi(bigrams)
    if not isinstance(stoi, dict):
        print(f"Expected stoi to be a dict")
        return
    s = sorted(stoi.keys())
    expect_eq("stoi keys when sorted", s, expected_s)
    expected_i = list(range(len(s)))
    i = sorted(stoi.values())
    expect_eq("stoi values when sorted", i, expected_i)
    print("get_stoi looks good. Onwards!")
test_get_stoi()

### Step 3: Map indices to characters

Objective: Write a function that takes the following arguments:
* a dict (```stoi```) as defined in step 2

And returns:
* a dict (```itos```) where ```itos``` contains the same key-value pairs as ```stoi``` but with keys and values swapped.

E.g. if ```stoi == {'.' : 0, 'b' : 1, 'z', 2}```, then ```itos == {0 : '.', 1 : 'b', 2 : 'z'}```

Video: [0:18:49](https://youtu.be/PaCmpygFfXo?t=1129)

In [ ]:
# TODO: Implement solution here

In [ ]:
import string

def test_get_itos():
    stoi = {elem:idx for idx, elem in enumerate(string.ascii_lowercase + ".")}
    itos = get_itos(stoi)
    if not isinstance(itos, dict):
        print(f"Expected stoi to be a dict")
        return
    for c in string.ascii_lowercase + ".":
        c_i = stoi[c]
        expect_eq(f"itos[{c_i}]", itos[c_i], c)
    print("get_itos looks good. Onwards!")
test_get_itos()

### Step 4: Split bigrams into inputs and outputs

Objective: Write a function that takes the following arguments:
* a list ```bigrams``` as defined in step 1, and
* a dict ```stoi``` as defined in step 2

And returns:
* a one-dimensional torch.Tensor ```x``` with all of the first characters in the tuples in ```bigrams```
* a one-dimensional torch.Tensor ```y``` with all of the second characters in the tuples in ```bigrams```
* Note: Both output tensors should be the same length as ```bigrams```

Video: [1:05:25](https://youtu.be/PaCmpygFfXo?t=3925)

In [ ]:
import torch

# TODO: Implement solution here

In [ ]:
def test_get_x_and_y():
    bigrams = [
        ('.', 'h'),
        ('h', 'i'),
        ('i', '.'),
        ('.', 'b'),
        ('b', 'y'),
        ('y', 'e'),
        ('e', '.'),
    ]
    stoi = {
        '.': 0,
        'h': 1,
        'i': 2,
        'b': 3,
        'y': 4,
        'e': 5,
    }
    x, y = get_x_and_y(bigrams, stoi)
    expect_eq("x[0]", x[0], 0)
    expect_eq("y[0]", y[0], 1)
    expect_eq("x[-2]", x[-2], 4)
    expect_eq("y[-2]", y[-2], 5)
    print("get_x_and_y looks good. Onwards!")
test_get_x_and_y()

### Step 5: Provide initial values for the model parameters

Objective: Write a function that takes the following arguments:
* a dict ```stoi``` as defined in step 2
  * the length of ```stoi``` will be referred to as ```stoi_n```
* a ```torch.Generator``` (```gen```) to provide (pseudo)random initial values for the parameters

And returns:
* a pytorch.Tensor ```W``` of shape (```stoi_n```, ```stoi_n```) where each element is randomly generated
* a pytorch.Tensor ```b``` of shape (1, ```stoi_n```)
  * The elements of ```b``` can be zero

Video: [1:14:03](https://youtu.be/PaCmpygFfXo?t=4433)

In [ ]:
import torch

# TODO: Implement solution here

In [ ]:
def test_initialize_w_b():
    stoi = {'q': 0, 'w': 1, 'e': 2, 'r': 3}
    expected_s_ct = 4
    gen = torch.Generator()
    gen.manual_seed(12345)
    W, b = initialize_w_b(stoi, gen)
    expect_eq("len(W)", len(W), expected_s_ct)
    for row_idx in range(len(W)):
        expect_eq(f"len(W[{row_idx}])", len(W[row_idx]), expected_s_ct)
        for col_idx in range(len(W[row_idx])):
            if (val := W[row_idx][col_idx]) == 0.0:
                print(f"Expected W[{row_idx}][{col_idx}] to be non-zero.")
                return
    expect_eq("W.requires_grad", W.requires_grad, True)
    expect_eq("b.requires_grad", b.requires_grad, True)
    expect_eq("b.shape", b.shape, (1, expected_s_ct))
    print("initialize_w_b looks good. Onwards!")
test_initialize_w_b()

### Step 6: Forward propagation

Objective: Write a function that takes the following arguments:
* a pytorch.Tensor ```x``` of training or testing inputs
* pytorch.Tensors ```W``` and ```b``` representing the parameters of the model

And returns:
* a pytorch.Tensor ```y_hat``` of the model's predicted outputs for each input in x
  * The predicted outputs for a given sample should sum to 1.0
  * The shape of ```y_hat``` should be (```len(x)```, ```len(W)```)
    * Note that ```len(W)``` represents the number of different characters in the word list

Video: [1:15:12](https://youtu.be/PaCmpygFfXo?t=4512)

In [ ]:
# TODO: Implement solution here

In [ ]:
def test_forward_prop():
    x = torch.tensor([
        1,
        0,
    ])

    W = torch.tensor([
        [0.1, 0.9, 0.2, 0.01],
        [0.04, 0.2, 1.6, 0.25],
        [0.02, 0.03, 0.7, 0.01],
    ], dtype=torch.float64)

    b = torch.tensor([
        0.01, 0.02, 0.03, 0.04
    ], dtype=torch.float64)

    expected_y_hat = torch.tensor([
        [0.1203, 0.1426, 0.5841, 0.1530],
        [0.1881, 0.4228, 0.2120, 0.1771],
    ], dtype=torch.float64)

    y_hat = forward_prop(x, W, b)

    if not torch.isclose(expected_y_hat, y_hat, rtol = 0.0, atol = 0.0001).all():
        print(f"Expected y_hat for test case to be \n{expected_y_hat}\n, was \n{y_hat}")
        return
    print("forward_prop looks good. Onwards!")
test_forward_prop()

### Step 7: Loss calculation
Objective: Write a function that takes the following arguments:
* a pytorch.Tensor ```y_hat``` of predicted outputs for a particular set of inputs
* a pytorch.Tensor ```y``` of actual outputs for the same set of inputs

And returns:
* a floating-point value representing the model's negative log likelihood loss for that set of inputs

Video: [1:35:49](https://youtu.be/PaCmpygFfXo&t=5749)

In [ ]:
# TODO: Implement solution here

In [ ]:
from math import exp

def test_calculate_loss():
    y = torch.tensor([2], dtype=torch.int64)
    y_hat = torch.tensor([
        [0.0, 0.0, 1.0, 0.0]
    ])
    if abs((loss := calculate_loss(y_hat, y))) > 0.00001:
        print(f"Expected loss for first example to be 0.0, was {loss}")
        return

    y = torch.tensor([2, 0], dtype=torch.int64)
    y_hat = torch.tensor([
        [0.09, 0.09, exp(-0.5), 0.09],
        [exp(-0.1), 0.01, 0.02, 0.03]
    ])
    if abs((loss := calculate_loss(y_hat, y)) - (expected_loss := 0.3)) > 0.00001:
        print(f"Expected loss for second example to be {expected_loss}, was {loss}")
        return
    print("calculate_loss looks good. Onwards!")
test_calculate_loss()

### Step 8: Gradient descent
Objective: Write a function that takes the following arguments:
* pytorch.Tensors ```W``` and ```b``` representing the parameters of the model
* a floating-point value ```learning_rate``` representing the overall size of adjustment to make to the parameters

And returns:
* the updated pytorch.Tensors ```W``` and ```b```
  * Note: Updating the parameters in-place is desirable, but for ease of testing, please return them regardless.

Video: [1:41:26](https://youtu.be/PaCmpygFfXo?t=6086)

In [ ]:
# TODO: Implement solution here

In [ ]:
def test_descend_gradient():
    W = torch.tensor([
        [1.0, 2.0,],
        [3.0, -4.0],
        [-5.0, 6.0],
    ])
    W.grad = torch.tensor([
        [-2.0, 1.0],
        [0.0, -2.0],
        [4.0, 1.0]
    ])
    b = torch.tensor([
        1.0,
        2.0,
    ])
    b.grad = torch.tensor([
        -1.0,
        0.5,
    ])
    new_w, new_b = descend_gradient(W, b, 3.0)
    expected_new_w = torch.tensor([
        [7.0, -1.0],
        [3.0, 2.0],
        [-17.0, 3.0]
    ])
    if not new_w.equal(expected_new_w):
        print(f"Expected new W for test case to be \n{expected_new_w}\n, is \n{new_w}")
        return
    expected_new_b = torch.tensor([
        4.0,
        0.5,
    ])
    if not new_b.equal(expected_new_b):
        print(f"Expected new b for test case to be \n{expected_new_b}\n, is \n{new_b}")
        return
    print("descend_gradient looks good. Onward!")
test_descend_gradient()

### Step 9: Train model
Objective: Write a function that takes the following arguments:
* pytorch.Tensors ```x``` and ```y``` as described in Step 4
* pytorch.Tensors ```W``` and ```b``` as described in Step 5
* a floating-point value ```learning_rate``` representing the overall size of adjustment to make to the parameters

Updates the values of W and b to fit the data slightly better

And returns:
* the loss as defined in Step 6

Implementation note: this function should make use of several of the functions you've previously implemented.

Video: [1:42:55](https://youtu.be/PaCmpygFfXo?t=6175)

In [ ]:
# TODO: Implement solution here

In [ ]:
def test_train_model():
    x = torch.tensor([
        0,
        1,
        2,
    ])
    y = torch.tensor([
        1,
        2,
        0,
    ])
    W = torch.tensor([
        [1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0],
    ], dtype=torch.float64, requires_grad=True)
    b = torch.tensor([
        0.1,
        0.2,
        0.3,
    ], dtype=torch.float64, requires_grad=True)

    loss = train_model(x, y, W, b, 2.0)

    expected_W = torch.tensor([
        [0.7996, 1.4452, 0.7552],
        [0.7996, 0.7785, 1.4219],
        [1.4663, 0.7785, 0.7552]
    ], dtype=torch.float64)
    if not torch.isclose(expected_W, W, rtol = 0.0, atol = 0.0001).all():
        print(f"Expected W for test case to be \n{expected_W}\n, was \n{W}")
        return

    expected_b = torch.tensor([
        0.1654,
        0.2022,
        0.2323
    ], dtype=torch.float64)
    if not torch.isclose(expected_b, b, rtol = 0.0, atol = 0.0001).all():
        print(f"Expected b for test case to be \n{expected_b}\n, was \n{b}")
        return
    print("train_model looks good. Onward!")
test_train_model()

### Step 10: Generate words
Objective: Write a function that takes the following arguments:
* pytorch.Tensors ```W``` and ```b``` as described in Step 5
* a dict ```stoi``` as described in Step 2
* a dict ```itos``` as described in Step 3
* a torch.Generator to use for pseudorandom selection of elements

Repeatedly generates a probability distribution for the next letter to select given the last letter

And returns
* a string representing a word generated by repeatedly sampling the probability distribution

Video: [1:54:31](https://youtu.be/PaCmpygFfXo?t=6871)

In [ ]:
# TODO: Implement solution here

In [ ]:
def test_generate_word():
    stoi = {
        '.': 0,
        'o': 1,
        'n': 2,
        'w': 3,
        'a': 4,
        'r': 5,
        'd': 6,
    }
    stoi_n = len(stoi)
    itos = {v:k for k,v in stoi.items()}

    W = torch.zeros((stoi_n, stoi_n), dtype=torch.float64)
    b = torch.zeros((1, stoi_n), dtype=torch.float64)
    # These weights result in a probability distribution where the desired next letter is roughly
    # 1000x as likely as the others.
    for i in range(stoi_n - 1):
        W[i][i+1] = 10.0
    W[stoi_n - 1][0] = 10.0

    gen = torch.Generator()
    gen.manual_seed(2147476727)
    expect_eq("generate_word result for test case", generate_word(W, b, stoi, itos, gen), "onward")
    print(f"generate_word looks good. Onward!")
test_generate_word()

### Finale: Put it all together

Objective: Write (and call) a function that:
* generates the bigrams and character maps
* repeatedly trains the model until its loss is acceptably small
  * For reference, the "perfect" loss of the probability table approach is approximately 2.4241
* uses the model to generate some made-up names

In [ ]:
# TODO: Implement solution here